In [79]:
import pandas as pd
from pycaret.classification import *
import numpy as np

In [304]:
# caminho1 = 'C:/Users/thiag/OneDrive/Documentos/arquivos/backteste/'
# df = pd.read_excel(caminho1 + 'backtesteargentina_A.xlsx')

df = pd.read_csv('BASE_ESTATISTICA_ARKAD_03_07.csv')


liga = 'CYPRUS - FIRST DIVISION'
flt = df.League == liga
df = df[flt]


In [305]:
total = len(df.index)

total_70 = 70*total/100

total_30 = total_70 - 30

total_70 = round(total_70)

total_30 = round(total_30)

print(total)

683


In [275]:
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df1 = df.loc[df.index <= total_70]
df1 = df1[['p_H', 'p_D', 'p_A', 'CV_VG_H', 'CV_VG_A', 'coeficiente_pontos', 'cv_pontos','CV_HDA', 'LayHome', 'LayAway']]
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df2 = df.loc[(df.index <= total) & (df.index > total_70)]
df2 = df2[['p_H', 'p_D', 'p_A', 'CV_VG_H', 'CV_VG_A', 'coeficiente_pontos', 'cv_pontos','CV_HDA', 'LayHome', 'LayAway']]
df2.replace([np.inf, -np.inf], np.nan, inplace=True)

In [163]:
features = ['p_H', 'p_D', 'p_A', 'CV_VG_H', 'CV_VG_A', 'coeficiente_pontos', 'cv_pontos','CV_HDA']

setup(data = df1, target = 'LayHome', numeric_features = features, ignore_features = [x for x in df1.columns.to_list() if x not in features and x != 'LayHome'], verbose = False)
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.7368,0.8319,0.8271,0.7369,0.7735,0.4601,0.4776,0.0140
ridge,Ridge Classifier,0.7330,0.0000,0.8200,0.7358,0.7699,0.4517,0.4693,0.0120
lr,Logistic Regression,0.7293,0.8220,0.8205,0.7324,0.7681,0.4439,0.4598,0.0540
nb,Naive Bayes,0.7291,0.8225,0.8062,0.7356,0.7639,0.4459,0.4595,0.0160
qda,Quadratic Discriminant Analysis,0.7067,0.8179,0.7648,0.7235,0.7372,0.4035,0.4158,0.0160
ada,Ada Boost Classifier,0.7036,0.7462,0.7452,0.7315,0.7330,0.3980,0.4043,0.0320
gbc,Gradient Boosting Classifier,0.6994,0.7470,0.7371,0.7243,0.7257,0.3904,0.3971,0.0320
rf,Random Forest Classifier,0.6993,0.7732,0.7233,0.7381,0.7232,0.3905,0.3991,0.0630
lightgbm,Light Gradient Boosting Machine,0.6991,0.7606,0.7362,0.7274,0.7228,0.3892,0.4010,0.0210
knn,K Neighbors Classifier,0.6953,0.7520,0.7433,0.7098,0.7209,0.3829,0.3945,0.0170


LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                           priors=None, shrinkage=None, solver='svd',
                           store_covariance=False, tol=0.0001)

In [164]:
modelo = ['lightgbm','qda','dt', 'rf', 'et' , 'lda',  'ridge', 'lr','nb', 'svm' , 'dummy', 'knn']

#, 'knn', 'ridge', 'gbc', 'rf','et','lda','ada',
for i in modelo:
    Treino = create_model(i, verbose=False)
    df1 = predict_model(Treino, df2, verbose=False)

    stake = 1
    green = stake / (df.FT_Odd_H - 0.85)
    red = -stake

    df1.loc[(df1['prediction_label'] == 1) & (df1['LayHome'] == 1), 'Profit'] = green
    df1.loc[(df1['prediction_label'] == 1) & (df1['LayHome'] == 0), 'Profit'] = red
    df1.loc[(df1['prediction_label'] == 0) & (df1['LayHome'] == 1), 'Profit'] = 0
    df1.loc[(df1['prediction_label'] == 0) & (df1['LayHome'] == 0), 'Profit'] = 0

    filtro = df1.prediction_label == 1
    df0 = df1[filtro]

    # Ajustando o Índice
    df0.reset_index(inplace=True, drop=True)
    df0.index = df0.index.set_names(['Nº'])
    df0 = df0.rename(index=lambda x: x + 1)

    df0['Profit_acu'] = df0.Profit.cumsum()
    # df0.Profit_acu.plot()
    profit = round(df0.Profit_acu.tail(1).item(),2)
    ROI = round((df0.Profit_acu.tail(1)/len(df0)*100).item(),2)
    print("")
    print("Modelo: "+i)
    print("Profit:",profit,"stakes em", len(df0),"jogos")
    print("ROI:",ROI,"%")



Modelo: lightgbm
Profit: 41.25 stakes em 107 jogos
ROI: 38.55 %

Modelo: qda
Profit: 55.12 stakes em 107 jogos
ROI: 51.51 %

Modelo: dt
Profit: 33.14 stakes em 99 jogos
ROI: 33.47 %

Modelo: rf
Profit: 39.56 stakes em 107 jogos
ROI: 36.97 %

Modelo: et
Profit: 37.58 stakes em 106 jogos
ROI: 35.45 %

Modelo: lda
Profit: 56.18 stakes em 116 jogos
ROI: 48.43 %

Modelo: ridge
Profit: 54.92 stakes em 110 jogos
ROI: 49.93 %

Modelo: lr
Profit: 52.37 stakes em 100 jogos
ROI: 52.37 %

Modelo: nb
Profit: 53.76 stakes em 98 jogos
ROI: 54.86 %

Modelo: svm
Profit: 48.13 stakes em 97 jogos
ROI: 49.62 %

Modelo: dummy
Profit: 24.5 stakes em 163 jogos
ROI: 15.03 %

Modelo: knn
Profit: 37.7 stakes em 95 jogos
ROI: 39.69 %


In [168]:
lda = create_model('lda', fold=10)

final_lda = finalize_model(lda)
save_model(final_lda, 'LAYHOME_ARKAD_'+liga)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8148,0.8824,0.9412,0.8000,0.8649,0.5768,0.5963
1,0.7037,0.7412,0.8824,0.7143,0.7895,0.3077,0.3280
2,0.8148,0.7588,0.9412,0.8000,0.8649,0.5768,0.5963
3,0.7037,0.7235,0.8824,0.7143,0.7895,0.3077,0.3280
4,0.4815,0.5926,0.6111,0.6111,0.6111,-0.1667,-0.1667
5,0.6667,0.7037,0.7222,0.7647,0.7429,0.2703,0.2712
6,0.6923,0.6993,0.8824,0.7143,0.7895,0.2409,0.2604
7,0.7308,0.8235,0.8824,0.7500,0.8108,0.3546,0.3690
8,0.8077,0.9346,0.8235,0.8750,0.8485,0.5860,0.5880


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=C:\Users\thiag\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['p_H', 'p_D', 'p_A', 'CV_VG_H',
                                              'CV_VG_A', 'coeficiente_pontos',
                                              'cv_pontos', 'CV_HDA'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               missing_values=nan,
                                                               strategy='mean',
                                                               verbose='deprecated...
                                                               fill_value='constant',
                                  

In [306]:
df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)
df1 = df.loc[df.index <= total_70]
df1 = df1[['p_H', 'p_D', 'p_A', 'CV_VG_H', 'CV_VG_A', 'coeficiente_pontos', 'cv_pontos','CV_HDA', 'LayHome', 'LayAway']]
df1.replace([np.inf, -np.inf], np.nan, inplace=True)
df2 = df.loc[(df.index <= total) & (df.index > total_70)]
df2 = df2[['p_H', 'p_D', 'p_A', 'CV_VG_H', 'CV_VG_A', 'coeficiente_pontos', 'cv_pontos','CV_HDA', 'LayHome', 'LayAway']]
df2.replace([np.inf, -np.inf], np.nan, inplace=True)

In [307]:
features = ['p_H', 'p_D', 'p_A', 'CV_VG_H', 'CV_VG_A', 'coeficiente_pontos', 'cv_pontos','CV_HDA']

setup(data = df1, target = 'LayAway', numeric_features = features, ignore_features = [x for x in df1.columns.to_list() if x not in features and x != 'LayAway'], verbose = False)
compare_models() 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.7120,0.0000,0.9079,0.7352,0.8109,0.2289,0.2425,0.0110
lr,Logistic Regression,0.7032,0.7071,0.9211,0.7218,0.8085,0.1867,0.2093,0.0560
et,Extra Trees Classifier,0.6940,0.6757,0.8498,0.7385,0.7886,0.2351,0.2513,0.0530
rf,Random Forest Classifier,0.6912,0.6567,0.8680,0.7294,0.7916,0.2071,0.2243,0.0590
lda,Linear Discriminant Analysis,0.6851,0.6993,0.8814,0.7210,0.7910,0.1697,0.1834,0.0140
gbc,Gradient Boosting Classifier,0.6824,0.6320,0.8366,0.7350,0.7809,0.2074,0.2160,0.0330
dummy,Dummy Classifier,0.6797,0.5000,1.0000,0.6797,0.8092,0.0000,0.0000,0.0130
nb,Naive Bayes,0.6791,0.7063,0.7581,0.7711,0.7603,0.2663,0.2723,0.0140
lightgbm,Light Gradient Boosting Machine,0.6738,0.6270,0.8014,0.7423,0.7684,0.2126,0.2201,0.0210
qda,Quadratic Discriminant Analysis,0.6734,0.6811,0.7980,0.7455,0.7680,0.2113,0.2117,0.0130


RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize='deprecated', positive=False,
                random_state=3416, solver='auto', tol=0.001)

In [308]:
modelo = ['lightgbm','qda','dt', 'rf', 'et' , 'lda',  'ridge', 'lr','nb', 'svm' , 'dummy', 'knn']

for i in modelo:
    Treino = create_model(i, verbose=False)
    df1 = predict_model(Treino, df2, verbose=False)

    stake = 1
    green = stake / (df.FT_Odd_A - 0.85)
    red = -stake

    df1.loc[(df1['prediction_label'] == 1) & (df1['LayAway'] == 1), 'Profit'] = green
    df1.loc[(df1['prediction_label'] == 1) & (df1['LayAway'] == 0), 'Profit'] = red
    df1.loc[(df1['prediction_label'] == 0) & (df1['LayAway'] == 1), 'Profit'] = 0
    df1.loc[(df1['prediction_label'] == 0) & (df1['LayAway'] == 0), 'Profit'] = 0

    filtro = df1.prediction_label == 1
    df0 = df1[filtro]

    # Ajustando o Índice
    df0.reset_index(inplace=True, drop=True)
    df0.index = df0.index.set_names(['Nº'])
    df0 = df0.rename(index=lambda x: x + 1)

    df0['Profit_acu'] = df0.Profit.cumsum()
    profit = round(df0.Profit_acu.tail(1).item(),2)
    ROI = round((df0.Profit_acu.tail(1)/len(df0)*100).item(),2)
    print("")
    print("Modelo: "+i)
    print("Profit:",profit,"stakes em", len(df0),"jogos")
    print("ROI:",ROI,"%")



Modelo: lightgbm
Profit: 35.07 stakes em 150 jogos
ROI: 23.38 %

Modelo: qda
Profit: 36.25 stakes em 140 jogos
ROI: 25.9 %

Modelo: dt
Profit: 21.01 stakes em 129 jogos
ROI: 16.28 %

Modelo: rf
Profit: 35.12 stakes em 157 jogos
ROI: 22.37 %

Modelo: et
Profit: 31.27 stakes em 154 jogos
ROI: 20.3 %

Modelo: lda
Profit: 38.37 stakes em 170 jogos
ROI: 22.57 %

Modelo: ridge
Profit: 38.62 stakes em 169 jogos
ROI: 22.85 %

Modelo: lr
Profit: 37.61 stakes em 172 jogos
ROI: 21.87 %

Modelo: nb
Profit: 40.83 stakes em 135 jogos
ROI: 30.25 %

Modelo: svm
Profit: 28.52 stakes em 204 jogos
ROI: 13.98 %

Modelo: dummy
Profit: 27.52 stakes em 205 jogos
ROI: 13.42 %

Modelo: knn
Profit: 36.04 stakes em 151 jogos
ROI: 23.87 %


In [309]:
nb = create_model('nb', fold=10)

final_nb = finalize_model(nb)
save_model(final_nb, 'LAYAWAY_ARKAD_'+liga)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6765,0.7194,0.6522,0.8333,0.7317,0.3392,0.3556
1,0.7647,0.7787,0.8261,0.8261,0.8261,0.4625,0.4625
2,0.7353,0.7945,0.7826,0.8182,0.8000,0.4093,0.4102
3,0.7353,0.7747,0.7391,0.8500,0.7907,0.4354,0.4433
4,0.6667,0.5696,0.8261,0.7308,0.7755,0.1378,0.1417
5,0.7273,0.7957,0.7826,0.8182,0.8000,0.3721,0.3730
6,0.5758,0.6391,0.6087,0.7368,0.6667,0.0977,0.1011
7,0.5152,0.5165,0.5909,0.6500,0.6190,-0.0435,-0.0439
8,0.7576,0.8058,0.9091,0.7692,0.8333,0.4000,0.4193


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=C:\Users\thiag\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['p_H', 'p_D', 'p_A', 'CV_VG_H',
                                              'CV_VG_A', 'coeficiente_pontos',
                                              'cv_pontos', 'CV_HDA'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               missing_values=nan,
                                                               strategy='mean',
                                                               verbose='deprecated...
                  TransformerWrapper(exclude=None, include=[],
                                     transformer=SimpleIm